In [1]:
import pandas as pd
import numpy as np
import gffutils
from collections import defaultdict
import warnings
import sys
import time
import threading
import glob

In [2]:
def rolling_progress(message, stop_event):
    """
    Displays a rolling progress indicator in the terminal while a task is running.

    Args:
        message (str): The message to display before the indicator.
        stop_event (threading.Event): An event to signal when to stop the animation.
    """
    symbols = ['|', '/', '-', '\\']  # Spinning symbols
    sys.stdout.write(message)  
    sys.stdout.flush()
    
    i = 0
    while not stop_event.is_set():  # Keep updating until stop_event is triggered
        sys.stdout.write(f"\b{symbols[i % len(symbols)]}")  # Overwrite last character
        sys.stdout.flush()
        time.sleep(0.2)  # Update every 0.2 seconds
        i += 1
    
    sys.stdout.write("\b Done! :)\n")  # Replace spinner with a checkmark
    sys.stdout.flush()


In [3]:
def calculate_transcript_features(grouped_features, transcript_data):
    '''
    Compute transcript features: transcript length, CDS length, and UTR lengths.

    Args:
        grouped_features (dict): Dictionary of transcript IDs mapped to their genomic features.
        transcript_data (DataFrame): DataFrame containing transcript data (transcript_ids, gene_names, transcript_types).

    Returns:
        DataFrame: Updated transcript metadata with computed feature lengths.
    '''

    transcript_features = []  # Store the results for all transcripts
    exons = defaultdict(list)  # Store exon features dictionary

    for tx_id, features in grouped_features.items():
        cds_length = 0
        utr5_length = 0
        utr3_length = 0
        tx_length = 0
        start_codon_pos = None
        stop_codon_pos = None

        # Separate features by type and store exon features in a dictionary
        cds_features = [f for f in features if f.featuretype == "CDS"]
        utr_features = [f for f in features if f.featuretype == "UTR"]
        exons[tx_id] = [f for f in features if f.featuretype == "exon"]

        # If no CDS features exist, set all lengths to 0， and compute transcript length by adding exons
        if not cds_features:
            cds_length = 0
            utr5_length = 0
            utr3_length = 0
            for exon in exons[tx_id]:
                tx_length += exon.end - exon.start + 1

        else:
            # Sort CDS features by start and end positions
            sorted_cds = sorted(cds_features, key=lambda x: (x.start, x.end))
            if utr_features[0].strand == "+":  # Positive strand
                start_codon_pos = sorted_cds[0].start  # First CDS start
                stop_codon_pos = sorted_cds[-1].end  # Last CDS end
            elif utr_features[0].strand == "-":  # Negative strand
                start_codon_pos = sorted_cds[-1].end  # Last CDS end
                stop_codon_pos = sorted_cds[0].start  # First CDS start

            # Calculate CDS length by summing the lengths of all CDS features
            for cds in sorted_cds:
                cds_length += cds.end - cds.start + 1

            # Handle UTRs and classify them as UTR5 or UTR3
            if utr_features:
                strand = utr_features[0].strand  # Assume all UTRs share the same strand
                for utr in utr_features:
                    utr_length = utr.end - utr.start + 1
                    if strand == "+":
                        if utr.end < start_codon_pos:  # 5' UTR is before start_codon
                            utr5_length += utr_length
                        elif utr.start > stop_codon_pos:  # 3' UTR is after stop_codon
                            utr3_length += utr_length
                    else:
                        if utr.start > start_codon_pos:  # 5' UTR is after start_codon on negative strand
                            utr5_length += utr_length
                        elif utr.end < stop_codon_pos:  # 3' UTR is before stop_codon on negative strand
                            utr3_length += utr_length
                
            tx_length = cds_length + utr3_length + utr5_length
    
        # Add the results for this transcript
        transcript_features.append({
            "transcript_id": tx_id,
            "tx_length": tx_length,
            "cds_length": cds_length,
            "utr5_length": utr5_length,
            "utr3_length": utr3_length
        })

    return pd.DataFrame(transcript_features).merge(transcript_data, on="transcript_id", how="left"), exons

# Function to convert transcript coordinates to genomic coordinates
def convert_to_genome_coordinates(tx_name, tx_pos, exonsdb, txfdb):
    """
    Convert transcript coordinates to genome coordinates and annotate regions.

    Args:
        tx_name (str): Transcript ID.
        tx_pos (int): Position in transcript coordinates.
        exonsdb (dict): Dictionary of transcript IDs mapped to exon features.
        txfdb (DataFrame): DataFrame containing transcript feature data.

    Returns:
        dict: Dictionary containing the genomic position, chromosome, exon junction distances, and region annotation.
    """
    
    if tx_name not in exonsdb:
        warnings.warn(f"Transcript {tx_name} not found in exons database.")
        return {
            "genome_pos": None,
            "chromosome": None,
            "dist_up_exon_junc": None,
            "dist_down_exon_junc": None,
            "region": None
        }
    
    tx_exons = sorted(exonsdb[tx_name], key=lambda x: x.start)  # Sort exons by start
    chrom = tx_exons[0].chrom
    strand = tx_exons[0].strand
    
    # Calculate cumulative exon lengths
    exon_lengths = [exon.end - exon.start + 1 for exon in tx_exons]
    cum_lengths = np.cumsum(exon_lengths)

    # Identify the exon containing the position
    idx = np.searchsorted(cum_lengths, tx_pos)
    
    if idx >= len(tx_exons):
        warnings.warn(f"Transcript position {tx_pos} is out of range for transcript {tx_name}")
        return {
            "genome_pos": None,
            "chromosome": None,
            "dist_up_exon_junc": None,
            "dist_down_exon_junc": None,
            "region": None
        }
    
    # Compute genomic position
    rel_pos = tx_pos - (cum_lengths[idx - 1] if idx > 0 else 0)
    exon = tx_exons[idx]
    if strand == "+":
        genome_pos = exon.start + rel_pos - 1
    else:
        genome_pos = exon.end - rel_pos + 1
    
    # Determine distance to exon junctions
    dist_upstream = rel_pos
    dist_downstream = exon.end - exon.start - rel_pos

    # Determine region (UTR, CDS, etc.)
    tx_feat = txfdb[txfdb["transcript_id"] == tx_name].iloc[0]
    if tx_feat["cds_length"] == 0:
        region = "ncRNA"
    elif tx_pos <= tx_feat["utr5_length"]:
        region = "UTR5"
    elif tx_pos <= tx_feat["utr5_length"] + tx_feat["cds_length"]:
        region = "CDS"
    elif tx_pos <= tx_feat["tx_length"]:
        region = "UTR3"
    else:
        region = None
    
    return {
        "genome_pos": genome_pos,
        "chromosome": chrom,
        "dist_up_exon_junc": dist_upstream,
        "dist_down_exon_junc": dist_downstream,
        "region": region
    }

def load_m6add_data(datatype):
    """
    Load and preprocess m6ADD dataset files.
    """
    files = glob.glob(f"GSE*_{datatype}.txt")
    dfs = []
    for f in files:
        df = pd.read_csv(f, sep="\t")
        dfs.append(df)
    df = pd.concat(dfs)
    df["chr"] = df["chr"].astype(str).str.lower().str.replace("chr", "")
    df["genome_pos"] = ((df["start"] + df["end"]) / 2).astype(int)
    return df

def fuzzy_merge(m6anet_df, m6add_df, window=3):
    """
    Perform fuzzy matching between m6anet and m6add dataframes based on genomic position.
    """
    matches = []
    for _, row in m6anet_df.iterrows():
        chr_match = m6add_df["chr"] == str(row["chromosome"])
        pos_match = (m6add_df["genome_pos"] >= row["genome_pos"] - window) & \
                    (m6add_df["genome_pos"] <= row["genome_pos"] + window)
        match_df = m6add_df[chr_match & pos_match]
        for _, match_row in match_df.iterrows():
            combined = row.to_dict()
            combined.update({f"m6add_{col}": match_row[col] for col in m6add_df.columns})
            matches.append(combined)
    return pd.DataFrame(matches)

In [26]:
input_file = "/home/nya0o0/hacks/project/m6anet_output/output.csv"
modification_sites = pd.read_csv(input_file)
modification_sites

,transcript_id,transcript_position,n_reads,probability_modified,kmer,mod_ratio
0,ENST00000393394.5,130,662,0.220579,GGACT,0.098187
1,ENST00000393394.5,234,441,0.247997,GGACT,0.090703
2,ENST00000222329.8,2631,23,0.354232,GGACT,0.130435
3,ENST00000523944.5,3348,30,0.556269,GGACT,0.233333
4,ENST00000262105.6,2540,20,0.390061,GGACT,0.200000
...,...,...,...,...,...,...
96,ENST00000634960.1,1360,24,0.655571,GGACT,0.333333
97,ENST00000634960.1,1394,28,0.242987,GGACT,0.071429
98,ENST00000634960.1,2171,29,0.233897,GGACT,0.103448
99,ENST00000626223.2,1417,62,0.591398,GGACT,0.193548


In [36]:
gtf_file = "/home/nya0o0/hacks/project/tests/data/gencode.v47.annotation.gtf"
db_file = gtf_file + ".db"
try: # Check whether the database already exists
    db = gffutils.FeatureDB(db_file, keep_order=True)
except:
    print("Creating GTF database...")
    db = gffutils.create_db(gtf_file, dbfn=db_file, force=True, keep_order=True, disable_infer_genes=True, disable_infer_transcripts=True)
    db = gffutils.FeatureDB(db_file, keep_order=True)
db

In [42]:
# Extract transcript IDs from the modification sites and remove version numbers
transcript_ids_to_keep = set(
    tid.split('.')[0] for tid in modification_sites["transcript_id"]
)

In [43]:
transcript_data = []
filtered_features = []
for feature in db.all_features():
    if "transcript_id" in feature.attributes:
        transcript_id = feature.attributes["transcript_id"][0].split(".")[0]
        if transcript_id in transcript_ids_to_keep:
            filtered_features.append(feature)  # Keep only relevant features
            if feature.featuretype == "transcript":
                transcript_data.append({
                    "transcript_id": feature.id.split(".")[0],
                    "gene_name": feature.attributes.get("gene_name", [None])[0],
                    "transcript_type": feature.attributes.get("transcript_type", [None])[0],
                })

In [44]:
# Turn the transcrip data to DataFrame and drop dupilcated data
transcript_data = pd.DataFrame(transcript_data).drop_duplicates("transcript_id")
transcript_data

,transcript_id,gene_name,transcript_type
0,ENST00000373062,GNL2,protein_coding
1,ENST00000368097,TAGLN2,protein_coding
2,ENST00000367627,TOR3A,protein_coding
3,ENST00000367142,NUCKS1,protein_coding
4,ENST00000258383,MRPL44,protein_coding
5,ENST00000326427,ITM2C,protein_coding
6,ENST00000232888,RRP9,protein_coding
7,ENST00000254799,GRSF1,protein_coding
8,ENST00000450253,EIF4E,protein_coding
9,ENST00000296412,ADH5,protein_coding
